# Streaming Twitter & Translator

## Build Streaming Twitter 

In [ ]:
## The following function is used for listening for twitter terms as well as translating text

In [6]:
## https://pythondata.com/collecting-storing-tweets-with-python-and-mongodb/

import pymongo
from pymongo import MongoClient
import tweepy
import json

# twitter account keys
CONSUMER_KEY = 'ENTER YOUR KEY HERE'
CONSUMER_SECRET = 'ENTER YOUR KEY HERE'
OAUTH_TOKEN = 'ENTER YOUR KEY HERE'
OAUTH_TOKEN_SECRET = 'ENTER YOUR KEY HERE'

WORDS = ['Hacking', 'Webshell','F5 Exploit','ZeroDay', '0day', 'WPA3 Exploit', 'router exploit', 'Novidade', 'VPNFilter', 'SMB Exploit', 'cisco exploit','استغلال F5', 'قرصنة الكمبيوتر','قذيفة الويب', 'استغلال سيسكو', 'Компьютерный Эксплоит', 'IoT Exploit']
class StreamListener(tweepy.StreamListener):    
    #This is a class provided by tweepy to access the Twitter Streaming API. 

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        #Connects to your mongoDB and stores the tweet
        try:
            client = MongoClient()
            
            # Use twitterdb database. If it doesn't exist, it will be created.
            db = client['Hacking_Tweets']
    
            # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            #grab the 'created_at' data from the Tweet to use for display
            created_at = datajson['created_at']

            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(created_at))
            
            #insert the data into the mongoDB into a collection called New_Project
            #if twitter_search doesn't exist, it will be created.
            db.New_Project.insert_one(datajson)
        except Exception as e:
           print(e)
        
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS)

Tracking: ['Hacking', 'Webshell', 'F5 Exploit', 'ZeroDay', '0day', 'WPA3 Exploit', 'router exploit', 'Novidade', 'VPNFilter', 'SMB Exploit', 'cisco exploit', 'استغلال F5', 'قرصنة الكمبيوتر', 'قذيفة الويب', 'استغلال سيسكو', 'Компьютерный Эксплоит', 'IoT Exploit']
You are now connected to the streaming API.
Tweet collected at Wed May 01 03:50:01 +0000 2019
Tweet collected at Wed May 01 03:50:04 +0000 2019
Tweet collected at Wed May 01 03:50:05 +0000 2019
Tweet collected at Wed May 01 03:50:05 +0000 2019
Tweet collected at Wed May 01 03:50:10 +0000 2019
Tweet collected at Wed May 01 03:50:14 +0000 2019
Tweet collected at Wed May 01 03:50:26 +0000 2019
Tweet collected at Wed May 01 03:50:31 +0000 2019
Tweet collected at Wed May 01 03:50:38 +0000 2019
Tweet collected at Wed May 01 03:50:40 +0000 2019
Tweet collected at Wed May 01 03:50:42 +0000 2019
Tweet collected at Wed May 01 03:50:46 +0000 2019
Tweet collected at Wed May 01 03:50:47 +0000 2019
Tweet collected at Wed May 01 03:50:48 +000

Tweet collected at Wed May 01 03:59:39 +0000 2019
Tweet collected at Wed May 01 03:59:42 +0000 2019
Tweet collected at Wed May 01 03:59:46 +0000 2019
Tweet collected at Wed May 01 03:59:49 +0000 2019
Tweet collected at Wed May 01 03:59:52 +0000 2019
Tweet collected at Wed May 01 03:59:55 +0000 2019
Tweet collected at Wed May 01 04:00:02 +0000 2019
Tweet collected at Wed May 01 04:00:03 +0000 2019
Tweet collected at Wed May 01 04:00:07 +0000 2019
Tweet collected at Wed May 01 04:00:09 +0000 2019
Tweet collected at Wed May 01 04:00:12 +0000 2019
Tweet collected at Wed May 01 04:00:17 +0000 2019
Tweet collected at Wed May 01 04:00:18 +0000 2019
Tweet collected at Wed May 01 04:00:22 +0000 2019
Tweet collected at Wed May 01 04:00:24 +0000 2019
Tweet collected at Wed May 01 04:00:29 +0000 2019
Tweet collected at Wed May 01 04:00:29 +0000 2019
Tweet collected at Wed May 01 04:00:30 +0000 2019
Tweet collected at Wed May 01 04:00:32 +0000 2019
Tweet collected at Wed May 01 04:00:32 +0000 2019


Tweet collected at Wed May 01 04:08:42 +0000 2019
Tweet collected at Wed May 01 04:08:47 +0000 2019
Tweet collected at Wed May 01 04:08:49 +0000 2019
Tweet collected at Wed May 01 04:08:52 +0000 2019
Tweet collected at Wed May 01 04:08:52 +0000 2019
Tweet collected at Wed May 01 04:08:58 +0000 2019
Tweet collected at Wed May 01 04:08:59 +0000 2019
Tweet collected at Wed May 01 04:09:02 +0000 2019
Tweet collected at Wed May 01 04:09:09 +0000 2019
Tweet collected at Wed May 01 04:09:12 +0000 2019
Tweet collected at Wed May 01 04:09:13 +0000 2019
Tweet collected at Wed May 01 04:09:16 +0000 2019
Tweet collected at Wed May 01 04:09:36 +0000 2019
Tweet collected at Wed May 01 04:09:36 +0000 2019
Tweet collected at Wed May 01 04:09:37 +0000 2019
Tweet collected at Wed May 01 04:09:37 +0000 2019
Tweet collected at Wed May 01 04:09:47 +0000 2019
Tweet collected at Wed May 01 04:09:47 +0000 2019
Tweet collected at Wed May 01 04:09:47 +0000 2019
Tweet collected at Wed May 01 04:09:49 +0000 2019


Tweet collected at Wed May 01 04:20:40 +0000 2019
Tweet collected at Wed May 01 04:20:44 +0000 2019
Tweet collected at Wed May 01 04:20:56 +0000 2019
Tweet collected at Wed May 01 04:21:00 +0000 2019
Tweet collected at Wed May 01 04:21:01 +0000 2019
Tweet collected at Wed May 01 04:21:06 +0000 2019
Tweet collected at Wed May 01 04:21:15 +0000 2019
Tweet collected at Wed May 01 04:21:18 +0000 2019
Tweet collected at Wed May 01 04:21:23 +0000 2019
Tweet collected at Wed May 01 04:21:25 +0000 2019
Tweet collected at Wed May 01 04:21:34 +0000 2019
Tweet collected at Wed May 01 04:21:50 +0000 2019
Tweet collected at Wed May 01 04:21:55 +0000 2019
Tweet collected at Wed May 01 04:21:59 +0000 2019
Tweet collected at Wed May 01 04:21:59 +0000 2019
Tweet collected at Wed May 01 04:21:59 +0000 2019
Tweet collected at Wed May 01 04:22:00 +0000 2019
Tweet collected at Wed May 01 04:22:00 +0000 2019
Tweet collected at Wed May 01 04:22:06 +0000 2019
Tweet collected at Wed May 01 04:22:08 +0000 2019


Tweet collected at Wed May 01 04:33:32 +0000 2019
Tweet collected at Wed May 01 04:33:50 +0000 2019
Tweet collected at Wed May 01 04:33:50 +0000 2019
Tweet collected at Wed May 01 04:33:52 +0000 2019
Tweet collected at Wed May 01 04:33:52 +0000 2019
Tweet collected at Wed May 01 04:33:54 +0000 2019
Tweet collected at Wed May 01 04:34:01 +0000 2019
Tweet collected at Wed May 01 04:34:02 +0000 2019
Tweet collected at Wed May 01 04:34:09 +0000 2019
Tweet collected at Wed May 01 04:34:19 +0000 2019
Tweet collected at Wed May 01 04:34:19 +0000 2019
Tweet collected at Wed May 01 04:34:28 +0000 2019
Tweet collected at Wed May 01 04:35:09 +0000 2019
Tweet collected at Wed May 01 04:35:11 +0000 2019
Tweet collected at Wed May 01 04:35:12 +0000 2019
Tweet collected at Wed May 01 04:35:25 +0000 2019
Tweet collected at Wed May 01 04:35:31 +0000 2019
Tweet collected at Wed May 01 04:35:33 +0000 2019
Tweet collected at Wed May 01 04:35:42 +0000 2019
Tweet collected at Wed May 01 04:35:47 +0000 2019


...

Tweet collected at Thu May 02 06:09:00 +0000 2019
Tweet collected at Thu May 02 06:09:04 +0000 2019
Tweet collected at Thu May 02 06:09:18 +0000 2019
Tweet collected at Thu May 02 06:09:24 +0000 2019
Tweet collected at Thu May 02 06:09:32 +0000 2019
Tweet collected at Thu May 02 06:09:42 +0000 2019
Tweet collected at Thu May 02 06:09:43 +0000 2019
Tweet collected at Thu May 02 06:09:49 +0000 2019
Tweet collected at Thu May 02 06:09:55 +0000 2019
Tweet collected at Thu May 02 06:10:12 +0000 2019
Tweet collected at Thu May 02 06:10:13 +0000 2019
Tweet collected at Thu May 02 06:10:23 +0000 2019
Tweet collected at Thu May 02 06:10:26 +0000 2019
Tweet collected at Thu May 02 06:10:28 +0000 2019
Tweet collected at Thu May 02 06:10:33 +0000 2019
Tweet collected at Thu May 02 06:10:51 +0000 2019
Tweet collected at Thu May 02 06:11:06 +0000 2019
Tweet collected at Thu May 02 06:11:14 +0000 2019
Tweet collected at Thu May 02 06:11:21 +0000 2019
Tweet collected at Thu May 02 06:11:22 +0000 2019


Tweet collected at Thu May 02 06:27:37 +0000 2019
Tweet collected at Thu May 02 06:27:41 +0000 2019
Tweet collected at Thu May 02 06:27:49 +0000 2019
Tweet collected at Thu May 02 06:27:51 +0000 2019
Tweet collected at Thu May 02 06:27:56 +0000 2019
Tweet collected at Thu May 02 06:28:00 +0000 2019
Tweet collected at Thu May 02 06:28:00 +0000 2019
Tweet collected at Thu May 02 06:28:00 +0000 2019
Tweet collected at Thu May 02 06:28:01 +0000 2019
Tweet collected at Thu May 02 06:28:08 +0000 2019
Tweet collected at Thu May 02 06:28:10 +0000 2019
Tweet collected at Thu May 02 06:28:14 +0000 2019
Tweet collected at Thu May 02 06:28:23 +0000 2019
Tweet collected at Thu May 02 06:28:28 +0000 2019
Tweet collected at Thu May 02 06:28:35 +0000 2019
Tweet collected at Thu May 02 06:28:42 +0000 2019
Tweet collected at Thu May 02 06:28:48 +0000 2019
Tweet collected at Thu May 02 06:28:57 +0000 2019
Tweet collected at Thu May 02 06:29:02 +0000 2019
Tweet collected at Thu May 02 06:29:06 +0000 2019


Tweet collected at Thu May 02 06:45:46 +0000 2019
Tweet collected at Thu May 02 06:45:52 +0000 2019
Tweet collected at Thu May 02 06:45:54 +0000 2019
Tweet collected at Thu May 02 06:46:10 +0000 2019
Tweet collected at Thu May 02 06:46:10 +0000 2019
Tweet collected at Thu May 02 06:46:17 +0000 2019
Tweet collected at Thu May 02 06:46:30 +0000 2019
Tweet collected at Thu May 02 06:46:49 +0000 2019
Tweet collected at Thu May 02 06:47:36 +0000 2019
Tweet collected at Thu May 02 06:47:45 +0000 2019
Tweet collected at Thu May 02 06:47:48 +0000 2019
Tweet collected at Thu May 02 06:48:00 +0000 2019
Tweet collected at Thu May 02 06:48:11 +0000 2019
Tweet collected at Thu May 02 06:48:14 +0000 2019
Tweet collected at Thu May 02 06:48:19 +0000 2019
Tweet collected at Thu May 02 06:48:28 +0000 2019
Tweet collected at Thu May 02 06:49:01 +0000 2019
Tweet collected at Thu May 02 06:49:03 +0000 2019
Tweet collected at Thu May 02 06:49:04 +0000 2019
Tweet collected at Thu May 02 06:49:17 +0000 2019


Tweet collected at Thu May 02 07:05:15 +0000 2019
Tweet collected at Thu May 02 07:05:21 +0000 2019
Tweet collected at Thu May 02 07:05:28 +0000 2019
Tweet collected at Thu May 02 07:05:45 +0000 2019
Tweet collected at Thu May 02 07:06:13 +0000 2019
Tweet collected at Thu May 02 07:06:33 +0000 2019
Tweet collected at Thu May 02 07:06:39 +0000 2019
Tweet collected at Thu May 02 07:06:44 +0000 2019
Tweet collected at Thu May 02 07:06:44 +0000 2019
Tweet collected at Thu May 02 07:06:51 +0000 2019
Tweet collected at Thu May 02 07:07:11 +0000 2019
Tweet collected at Thu May 02 07:07:15 +0000 2019
Tweet collected at Thu May 02 07:07:16 +0000 2019
Tweet collected at Thu May 02 07:07:23 +0000 2019
Tweet collected at Thu May 02 07:07:23 +0000 2019
Tweet collected at Thu May 02 07:07:30 +0000 2019
Tweet collected at Thu May 02 07:07:30 +0000 2019
Tweet collected at Thu May 02 07:07:35 +0000 2019
Tweet collected at Thu May 02 07:07:35 +0000 2019
Tweet collected at Thu May 02 07:07:36 +0000 2019


KeyboardInterrupt: 

## Build Translation Function

In [ ]:
### Text Translation Function

In [ ]:
# The following code for tranlation was borrowed from: https://github.com/mouuff/mtranslate/blob/master/mtranslate/core.py

import sys
import re

if (sys.version_info[0] < 3):
    #pythong 2 syntax
    import urllib2
    import urllib
    import HTMLParser
else:
    #python 3 syntax
    import html.parser
    import urllib.request
    import urllib.parse

#this is the user agent string google will see as it is interpreting the tweets    
agent = {'User-Agent':
"Mozilla/4.0 (\
compatible;\
MSIE 6.0;\
Windows NT 5.1;\
SV1;\
.NET CLR 1.1.4322;\
.NET CLR 2.0.50727;\
.NET CLR 3.0.04506.30\
)"}

#these functions are designed to identify if python 2 or python 3 is being used. 
def unescape(text):
    if (sys.version_info[0] < 3):
        parser = HTMLParser.HTMLParser() #python 2
    else:
        parser = html.parser.HTMLParser() #python 3
    return (parser.unescape(text))

def translate(to_translate, to_language="auto", from_language="auto"):
    """Returns the translation using google translate
    you must shortcut the language you define
    (French = fr, English = en, Spanish = es, etc...)
    if not defined it will detect it or use english by default
    Example:
    print(translate("salut tu vas bien?", "en"))
    hello you alright?
    """
    base_link = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"
    if (sys.version_info[0] < 3):
        to_translate = urllib.quote_plus(to_translate)
        link = base_link % (to_language, from_language, to_translate)
        request = urllib2.Request(link, headers=agent)
        raw_data = urllib2.urlopen(request).read()
    else:
        to_translate = urllib.parse.quote(to_translate)
        link = base_link % (to_language, from_language, to_translate)
        request = urllib.request.Request(link, headers=agent)
        raw_data = urllib.request.urlopen(request).read()
    data = raw_data.decode("utf-8")
    expr = r'class="t0">(.*?)<'
    re_result = re.findall(expr, data)
    if (len(re_result) == 0):
        result = ""
    else:
        result = unescape(re_result[0])
    return (result)